In [ ]:
# | default_exp buildingediting

In [ ]:
# | include: false
from nbdev.showdoc import *

%load_ext autoreload
%autoreload 2

In [ ]:
# | export
import cobra

In [ ]:
# | export
def add_metabolite(model, name, compartment="default", multi=True):
    """
    This function adds a metabolite to the given model, and returns this metabolite.
    If multi is set to true it will be added to every phase of the model.
    """
    if multi == True:
        for i in range(1, check_number_of_models(model) + 1):
            new_name = name + "_" + str(i)
            metabolite = cobra.core.Metabolite(new_name)
            metabolite.id = new_name
            metabolite.name = new_name
            metabolite.compartment = compartment
            model.add_metabolites([metabolite])
    else:
        metabolite = cobra.core.Metabolite(name)
        metabolite.id = name
        metabolite.name = name
        metabolite.compartment = compartment
        model.add_metabolites([metabolite])

    return metabolite

In [ ]:
show_doc(add_metabolite)

<h4 id="add_metabolite" class="doc_header"><code>add_metabolite</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>add_metabolite</code>(**`model`**, **`name`**, **`compartment`**=*`'default'`*, **`multi`**=*`True`*)

This function adds a metabolite to the given model, and returns this metabolite.
If multi is set to true it will be added to every phase of the model.

In [ ]:
# | export
def add_reaction(model, name, multi=True):
    """
    This function adds a reaction to the given model, and returns this reaction.
    If multi is set to true it will be added to every phase of the model.
    """
    if multi == True:
        for i in range(1, check_number_of_models(model) + 1):
            new_name = name + "_" + str(i)
            reaction = cobra.core.Reaction(new_name)
            reaction.id = new_name
            reaction.name = new_name
            model.add_reactions([reaction])
    else:
        reaction = cobra.core.Reaction(name)
        reaction.id = name
        reaction.name = name
        model.add_reactions([reaction])
    return reaction

In [ ]:
show_doc(add_reaction)

<h4 id="add_reaction" class="doc_header"><code>add_reaction</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>add_reaction</code>(**`model`**, **`name`**, **`multi`**=*`True`*)

This function adds a reaction to the given model, and returns this reaction.
If multi is set to true it will be added to every phase of the model.

In [ ]:
# | export
def split_model(model, labels):
    """
    This function takes two arguments: the model to be split and the labels for the models to split it into.
    len(labels) number of models will be created, with the suffix "_label" to define them.
    """

    number_of_models = len(labels)
    old_model = model.copy()

    # Create new metabolites for models
    old_metabolites = []
    for i in range(len(model.metabolites)):
        old_metabolite = model.metabolites[i]
        for j in labels:
            new_metabolite = old_metabolite.copy()
            new_metabolite.name = new_metabolite.name + "_" + str(j)
            new_metabolite.id = new_metabolite.id + "_" + str(j)
            model.add_metabolites(new_metabolite)
        old_metabolites.append(old_metabolite)

    # Create new reactions for models
    old_reactions = []
    for i in range(len(model.reactions)):
        old_reaction = model.reactions[i]
        for j in labels:
            new_reaction = old_reaction.copy()
            new_reaction.name = new_reaction.name + "_" + str(j)
            new_reaction.id = new_reaction.id + "_" + str(j)
            new_metabolites = {}
            for metabolite, value in new_reaction.metabolites.items():
                new_metabolites[str(metabolite) + "_" + str(j)] = value
            model.add_reactions([new_reaction])
            new_reaction = model.reactions.get_by_id(new_reaction.id)
            new_reaction.subtract_metabolites(new_reaction.metabolites)
            new_reaction.add_metabolites(new_metabolites)
        old_reactions.append(old_reaction)

    # Remove old reactions and metabolites
    model.remove_reactions(old_reactions)
    model.remove_metabolites(old_metabolites)

    # tests

    # Check no. of metabolites and reactions is equal
    if (
        len(model.reactions) == number_of_models * len(old_model.reactions)
        and len(model.metabolites) == number_of_models * len(old_model.metabolites)
    ) == False:
        raise UserWarning("Number of metabolites and reactants does not match")

    empty_reactions = []
    for reaction in model.reactions:
        if len(reaction.metabolites) == 0:
            empty_reactions.append(reaction.name)
    if len(empty_reactions) != 0:
        raise UserWarning(
            "There are some reactions with no metabolites, they are: "
            + str(emptyreactions)
        )

In [ ]:
show_doc(split_model)

<h4 id="split_model" class="doc_header"><code>split_model</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>split_model</code>(**`model`**, **`labels`**)

This function takes two arguments: the model to be split and the labels for the models to split it into.
len(labels) number of models will be created, with the suffix "_label" to define them.

In [ ]:
# | export
def check_number_of_models(model):
    """
    This function returns the number of phases of the model based on the maximal number of the last tag
    """
    submodel_numbers = set()
    for metabolite in model.metabolites:
        try:
            submodel_numbers.add(int(metabolite.name[-1:]))
        except:
            pass
    try:
        model_number = max(submodel_numbers)
    except:
        model_number = 1
    return model_number

In [ ]:
show_doc(check_number_of_models)

<h4 id="check_number_of_models" class="doc_header"><code>check_number_of_models</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>check_number_of_models</code>(**`model`**)

This function returns the number of phases of the model based on the maximal number of the last tag

In [ ]:
# | export
def set_bounds_multi(model, reaction_id, lower_flux, upper_flux):
    """
    Constrain a reaction's flux in all phases
    """

    for i in range(1, check_number_of_models(model) + 1):
        model.reactions.get_by_id(reaction_id + "_" + str(i)).bounds = (
            lower_flux,
            upper_flux,
        )

    return model

In [ ]:
show_doc(set_bounds_multi)

<h4 id="set_bounds_multi" class="doc_header"><code>set_bounds_multi</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>set_bounds_multi</code>(**`model`**, **`reaction_id`**, **`lower_flux`**, **`upper_flux`**)

Constrain a reaction's flux in all phases

In [ ]:
# | export
def add_linkers(model, linkers_file, compartments, cells, phase_times):
    """
    This function adds linker reactions for osmolytes to a phased model of guard cell, and adds osmolarity and charge linker pseudoreactions.
    It takes five arguments:
    model: the model to which linkers are to be added
    linkersfile: a .csv file with a list of osmolytes and their osmotic coefficients, leave the osmotic coefficient at 0 to add as a linker but not contribute to osmolarity
    compartments: a list of the compartments to add linker reactions to within cells
    cells: a list of cell tags to be used
    phase_times: a list of the lengths of each phase, for scaling of the linker reactions
    """

    number_of_models = check_number_of_models(model)
    linkers = def_linkers(linkers_file)
    cell_compartments = []
    for compartment in compartments:
        if compartment == "a":
            cell_compartments.append(compartment)
        else:
            for cell in cells:
                cell_compartments.append(compartment + "_" + cell)

    # create the linker reactions from the 'osmolytes' list
    new_reactions = []
    for linker in linkers:
        add_reaction(model, linker + "_Linker")

    # add pseudometabolites to represent accumulation reactions
    for compartment in cell_compartments:
        add_metabolite(model, "pseudoOs_" + compartment, compartment)
        add_metabolite(model, "pseudoCharge_" + compartment, compartment)

    # add metabolites to linker reactions which are scaled according to phase lengths
    for compartment in cell_compartments:
        for linker in linkers:
            if "_" + compartment in linker:
                for i in range(1, number_of_models):
                    model.reactions.get_by_id(
                        linker + "_Linker_" + str(i)
                    ).add_metabolites(
                        {
                            linker + "_" + str(i): (1.0 / phase_times[i - 1]) * -1,
                            linker + "_" + str(i + 1): (1.0 / phase_times[i]) * 1,
                            "pseudoOs_"
                            + compartment
                            + "_"
                            + str(i): int(
                                linkers[linker]
                            ),  # this is where the contribution is 0 if osmotic coeffient is set to 0
                            "pseudoCharge_"
                            + compartment
                            + "_"
                            + str(i): float(
                                model.metabolites.get_by_id(
                                    linker + "_" + str(i)
                                ).charge
                            )
                            * (1.0 / phase_times[i - 1])
                            * -1,
                        }
                    )
                model.reactions.get_by_id(
                    linker + "_Linker_" + str(number_of_models)
                ).add_metabolites(
                    {
                        linker
                        + "_"
                        + str(number_of_models): (
                            1.0 / phase_times[number_of_models - 1]
                        )
                        * -1,
                        linker + "_" + str(1): (1.0 / phase_times[0]) * 1,
                        "pseudoOs_"
                        + compartment
                        + "_"
                        + str(number_of_models): int(linkers[linker]),
                        "pseudoCharge_"
                        + compartment
                        + "_"
                        + str(number_of_models): float(
                            model.metabolites.get_by_id(linker + "_" + str(i)).charge
                        )
                        * (1.0 / phase_times[number_of_models - 1])
                        * -1,
                    }
                )

    # add osmotic constraint reactions

    add_reaction(model, f"pseudoOs_constraint_a", multi=True)
    add_metabolites_to_reaction_multi(model, f"pseudoOs_constraint_a", {
        f"pseudoOs_a": -1,
    })

    for cell in cells:
        for compartment in compartments:
            if compartment != "a":
                add_reaction(model, f"pseudoOs_constraint_{compartment}_{cell}", multi=True)
                add_metabolites_to_reaction_multi(model, f"pseudoOs_constraint_{compartment}_{cell}", {
                    f"pseudoOs_{compartment}_{cell}": -1,
                }
                )
    # tests

    # run a basic test to ensure new reactions have metabolites attached to them
    empty_reactions = []
    for reaction in model.reactions:
        if len(reaction.metabolites) == 0:
            empty_reactions.append(reaction.name)
    if len(empty_reactions) != 0:
        raise UserWarning(
            "There are some reactions with no metabolites, they are: "
            + str(empty_reactions)
        )

In [ ]:
show_doc(add_linkers)

<h4 id="add_linkers" class="doc_header"><code>add_linkers</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>add_linkers</code>(**`model`**, **`linkers_file`**, **`compartments`**, **`cells`**, **`phase_times`**)

This function adds linker reactions for osmolytes to a phased model of guard cell, and adds osmolarity and charge linker pseudoreactions.
It takes five arguments:
model: the model to which linkers are to be added
linkersfile: a .csv file with a list of osmolytes and their osmotic coefficients
compartments: a dictionary with the tags of compartments as keys and their relative volumes as values
cells: a list of cell tags to be used
phase_times: a list of the lengths of each phase, for scaling of the linker reactions

In [ ]:
# | export
def def_linkers(file):
    """
    This function creates a dict of osmolytes and their osmotic coefficients from the .csv file, and is used by addlinkers
    """
    import csv

    with open(file, mode="r") as csv_file:
        csv_linkers = csv.DictReader(csv_file)
        linkers = {}
        line_count = 0
        for linker in csv_linkers:
            if line_count == 0:
                line_count += 1
            linkers[linker["Linker"]] = linker["Osmotic Coefficient"]
            line_count += 1
    return linkers

In [ ]:
show_doc(def_linkers)

<h4 id="def_linkers" class="doc_header"><code>def_linkers</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>def_linkers</code>(**`file`**)

This function creates a dict of osmolytes and their osmotic coefficients from the .csv file, and is used by addlinkers

In [ ]:
# | export


def add_metabolites_to_reaction_multi(model, reaction, metabolites_dict):
    """
    This function adds a dictionary of metabolites to a reaction in every phase of the model
    """
    for i in range(1, check_number_of_models(model) + 1):
        for metabolite in metabolites_dict:
            model.reactions.get_by_id(f"{reaction}_{i}").add_metabolites(
                {f"{metabolite}_{i}": metabolites_dict[metabolite]}
            )

In [ ]:
show_doc(add_metabolites_to_reaction_multi)

<h4 id="add_metabolites_to_reaction_multi" class="doc_header"><code>add_metabolites_to_reaction_multi</code><a href="__main__.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>add_metabolites_to_reaction_multi</code>(**`model`**, **`reaction`**, **`metabolites_dict`**)

This function adds a dictionary of metabolites to a reaction in every phase of the model